# Inicjalizacja

In [ ]:
%matplotlib widget

from evaluator import evaluate, calc_impact, create_functions_in_db, calc_prs_bugginess, overwrite_bugginess_function, get_considered_prs
from definitions import Repository
from configuration import ProjectConfiguration
import db
import smells
import metrics
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import ipywidgets as widgets


#plots size in inches
plt.rcParams["figure.figsize"] = (9.75,5)

In [ ]:
config = ProjectConfiguration()

db.prepare(config.connstr)

dbsession = db.get_session()
#create_functions_in_db(dbsession)

# Ewaluacja

In [ ]:
repositories = list(map(lambda repository_name: dbsession.query(Repository).filter(Repository.full_name == repository_name).first(), config.projects))

if None in repositories:
    raise LookupError("One of repositories does not exist in the database")

## Udział smelly prs wśród badanych prs (reprodukcja)

In [ ]:
chosen_simple_tests = [
    smells.lack_of_review,
    smells.missing_description,
    smells.large_changesets,
    smells.sleeping_reviews,
    smells.review_buddies,
    smells.ping_pong
]
chosen_complex_tests = [
    (smells.union, [smells.lack_of_review,
                    smells.missing_description,
                    smells.large_changesets,
                    smells.sleeping_reviews,
                    smells.review_buddies,
                    smells.ping_pong]),
    (smells.intersection, [smells.lack_of_review,
                    smells.missing_description,
                    smells.large_changesets,
                    smells.sleeping_reviews,
                    smells.review_buddies,
                    smells.ping_pong])
]

In [ ]:
smells_evaluations = {}

for repository in repositories:
    tests_results = list(map(lambda simple_test: evaluate(repository.full_name, simple_test), chosen_simple_tests))
    tests_results.extend(list(map(lambda complex_test: evaluate(repository.full_name, complex_test[0], complex_test[1]), chosen_complex_tests)))
    smells_evaluations[repository.full_name] = tests_results

In [ ]:
# display results as a text
print("Smell / repository".ljust(30), end="\t")
column_width=max(len(repo.name) for repo in repositories)+1
print(*map(lambda repo: repo.name.ljust(column_width), repositories),sep="\t")

for i in range(0, len(chosen_simple_tests)):
    print(next(iter(smells_evaluations.values()))[i].evaluator_name.ljust(30), end="\t")
    print(*map(lambda tests_results: f"{round(tests_results[i].percentage*100,2)}%".rjust(column_width), smells_evaluations.values()), sep="\t")

for i in range(len(chosen_simple_tests), len(chosen_simple_tests)+len(chosen_complex_tests)):
    print(next(iter(smells_evaluations.values()))[i].evaluator_name.ljust(30), end="\t")
    print(*map(lambda tests_results: f"{round(tests_results[i].percentage*100,2)}%".rjust(column_width), smells_evaluations.values()), sep="\t")

In [ ]:
# display results as a plot
labels = list(map(lambda eval: eval.evaluator_name, next(iter(smells_evaluations.values()))))
x = np.arange(len(labels))
width = 0.35

fig, ax = plt.subplots()
counter = 0.5
for repository in smells_evaluations:
    bar = ax.bar(x - width/2 + width/len(smells_evaluations)*counter, list(map(lambda evaluation: evaluation.percentage, smells_evaluations[repository])), width/len(smells_evaluations), label=repository)
    counter+=1
ax.set_xticks(x, labels, rotation="vertical")
ax.set_ylim([0, 1])
ax.legend()
fig.tight_layout()
plt.show()

## Wpływ smelli na prawdopodobieństwo wprowadzenia błędu

In [ ]:
# tests_for_impact_evaluation = [
#     ("Lack of code review", smells.lack_of_review),
#     ("Missing PR description", smells.missing_description),
#     ("Large changeset", smells.large_changesets),
#     ("Sleeping reviews", smells.sleeping_reviews),
#     ("Review Buddies", smells.review_buddies),
#     ("Ping-pong reviews", smells.ping_pong)
# ]

In [ ]:
# impact_evaluations = {}

# for repository in repositories:
#     print(f"[{repository.full_name}] Started impact evaluation")
#     impact_evaluations[repository.full_name] = list(map(lambda pair: (pair[0], calc_impact(session = dbsession,
#                                                                  repo = dbsession.get(Repository, repository.id),
#                                                                  evaluator = pair[1],
#                                                                  evaluator_args=None)), tests_for_impact_evaluation))
#     print(f"[{repository.full_name}] Finished impact evaluation")

In [ ]:
# # display results as text
# print(
#     f"Percentage of pulls where at least one file was changed next by bug solving PR:")

# print("Smell / repository".ljust(30), end="\t")
# column_width = max(column_width, 22)
# print(*map(lambda repo: repo.name.ljust(column_width), repositories),sep="\t")

# for i in range(0, len(tests_for_impact_evaluation)):
#     print(next(iter(impact_evaluations.values()))[i][0].ljust(30), end="\t")
#     print(*map(lambda tests_results: (f"+{round(tests_results[i][1][0]*100,1)}% ".rjust(7)+
#                                      f"-{round(tests_results[i][1][1]*100,1)}% ".rjust(7)+"Δ="+
#                                      (('+' if tests_results[i][1][1]>tests_results[i][1][0] else '')+
#                                      f"{round((tests_results[i][1][1]-tests_results[i][1][0])*100,1)}%").rjust(6)).rjust(column_width), impact_evaluations.values()), sep="\t")

In [ ]:
# # display results as a plot
# labels = list(map(lambda eval: eval[0], next(iter(impact_evaluations.values()))))
# x = np.arange(len(labels))

# width=0.35
# fig, ax = plt.subplots()
# counter = 0.5
# for repository in impact_evaluations:
#     bar = ax.bar(x - width/2 + width/len(impact_evaluations)*counter, list(map(lambda evaluation: evaluation[1][1]-evaluation[1][0], impact_evaluations[repository])), width/len(smells_evaluations), label=repository)
#     counter+=1
# ax.set_xticks(x, labels, rotation="vertical")
# ax.set_ylim([-1, 1])
# ax.legend()
# plt.gcf().subplots_adjust(bottom=0.35)
# plt.show()

## Metryki

In [ ]:
calculated_metrics = [
    metrics.review_window_metric,
    metrics.review_window_per_line_metric
    #metrics.review_chars_code_lines_ratio,
    #metrics.reviewed_lines_per_hour
]

In [ ]:
from sqlalchemy.orm import joinedload, subqueryload
from definitions import PullRequest, Commit
from sqlalchemy import select, column

metrics_evaluations = {}

for repository in repositories:
    print(f"[{repository.full_name}] Preparing queries")
    for metric in calculated_metrics:
        metrics_evaluations[repository.full_name] = metrics_evaluations.get(repository.full_name, [])
        metrics_evaluations[repository.full_name].append(evaluate(repository.full_name, metric))
    metrics_evaluations[repository.full_name] = metrics_evaluations.get(repository.full_name, [])

# create dataframes
metrics_evaluations_df = {}
for repository in repositories:
    print(f"[{repository.full_name}] Creating dataframe")
    df = pd.DataFrame()
    df["pull_id"] = list(map(lambda r: float(r[0]) if r[0] is not None else None, dbsession.execute(select(column("id")).select_from(metrics_evaluations[repository.full_name][0].evaluated.subquery())).all()))
    for m in metrics_evaluations[repository.full_name]:
        print(f"[{repository.full_name}] Creating dataframe column {m.metric_name}")
        df[m.metric_name] = m.to_list(dbsession)
        
    # add buggy
    df_buggy = pd.DataFrame()
    df_buggy["buggy"] = -1
    for index, row in df.iterrows():
        df_buggy = pd.concat([df_buggy, pd.DataFrame({"buggy": [int(any(list(map(lambda commit: commit.buggy, dbsession.query(PullRequest).get(row["pull_id"]).commits))))]}, [index])])
    df = df.join(df_buggy["buggy"])
    
    metrics_evaluations_df[repository.full_name] = df


print("[All repositories] Creating common dataframe")
overall_metrics_df = pd.concat(metrics_evaluations_df.values())
print(f"[All repositories] Finished creating dataframes")

In [ ]:
# calculate correlation
def calc_correlations(method):
    correlations = {}
    for repository in repositories:
        correlations[repository.full_name] = metrics_evaluations_df[repository.full_name].corr(method=method)
    overall_correlations = overall_metrics_df.corr(method=method)
    return correlations, overall_correlations

In [ ]:
# display correlation heatmaps
@widgets.interact(corr_method=["pearson", "kendall", "spearman"])
def display_corr_plots(corr_method="spearman"):
    correlations, overall_correlations = calc_correlations(corr_method)
    for repository in repositories:
        fig, ax = plt.subplots()
        hm = sns.heatmap(correlations[repository.full_name], annot = True, ax=ax)
        hm.set(title = f"Correlation matrix of reviews metrics and bugginess for {repository.full_name}\n")
        fig.subplots_adjust(left=0.2, bottom=0.4)
        ax.set_xticklabels(list(map(lambda label: label.get_text().replace('_',' '), ax.get_xticklabels())),rotation="vertical")
        ax.set_yticklabels(list(map(lambda label: label.get_text().replace('_',' '), ax.get_yticklabels())))
        plt.show()
    fig, ax = plt.subplots()
    hm = sns.heatmap(overall_correlations, annot = True, ax=ax)
    hm.set(title = f"Correlation matrix of reviews metrics and bugginess for all repositories\n")
    fig.subplots_adjust(left=0.2, bottom=0.4)
    ax.set_xticklabels(list(map(lambda label: label.get_text().replace('_',' '), ax.get_xticklabels())),rotation="vertical")
    ax.set_yticklabels(list(map(lambda label: label.get_text().replace('_',' '), ax.get_yticklabels())))
    plt.show()

In [ ]:
# display boxplots
@widgets.interact(outlier=False)
def display_boxplots(outlier):
    for column_name in overall_metrics_df.columns[1:-1]:
        for repository in repositories:
            fig, ax = plt.subplots()
            df = metrics_evaluations_df[repository.full_name]
            ax.boxplot([df[df.buggy == 0][column_name], df[df.buggy == 1][column_name]], 0, 'k+' if outlier else '')
            ax.set_xticklabels(["nonbuggy", "buggy"], fontsize=8)
            ax.set_title(f"Boxplot for {column_name.replace('_',' ')} in repository {repository.full_name}")
            plt.show()
        fig, ax = plt.subplots()
        df = overall_metrics_df
        col = getattr(df, column_name)
        ax.boxplot([df[df.buggy == 0][column_name], df[df.buggy == 1][column_name]], 0, 'k+' if outlier else '')
        ax.set_xticklabels(["nonbuggy", "buggy"], fontsize=8)
        ax.set_title(f"Boxplot for {column_name.replace('_',' ')} in all repositories")
        plt.show()

## Model

### Preparing data

In [ ]:
df = overall_metrics_df.copy()
labels = np.array(df["buggy"])
features = df.drop("pull_id", axis = 1)
features = features.drop("buggy", axis = 1)
feature_list = list(features.columns)
features = np.array(features)


### Preparing training and testing sets

In [ ]:
from sklearn.model_selection import train_test_split

train_features, test_features, train_labels, test_labels = train_test_split(features, labels, test_size = 0.25, random_state = 42)

print('Training Features Shape:', train_features.shape)
print('Training Labels Shape:', train_labels.shape)
print('Testing Features Shape:', test_features.shape)
print('Testing Labels Shape:', test_labels.shape)

In [ ]:
# baseline error should go here

### Model training

In [ ]:
from sklearn.ensemble import RandomForestRegressor

rf = RandomForestRegressor(n_estimators = 10000, random_state = 42, max_depth=1000)
rf.fit(train_features, train_labels);

### Predicting

In [ ]:
# Use the forest's predict method on the test data
predictions = rf.predict(test_features)

### Performance check

In [ ]:
# Calculate the absolute errors
errors = abs(predictions - test_labels)
# Print out the mean absolute error (mae)
print(f"Mean Absolute Error: {round(np.mean(errors), 2)}%")

### Metrics importance

In [ ]:
# Get numerical feature importances
importances = list(rf.feature_importances_)
# List of tuples with variable and importance
feature_importances = [(feature, round(importance, 2)) for feature, importance in zip(feature_list, importances)]
# Sort the feature importances by most important first
feature_importances = sorted(feature_importances, key = lambda x: x[1], reverse = True)
# Print out the feature and importances
print("Metrics                        Importance")
[print(f"{pair[0].replace('_',' ').ljust(30,' ')} {pair[1]}") for pair in feature_importances];

In [ ]:
fig, ax = plt.subplots()
ax.bar(feature_list, importances)
ax.set_xticklabels(feature_list)
ax.set_ylim(0,1)
plt.show()

### Example of a tree

In [ ]:
from sklearn import tree
# Pull out one tree from the forest
@widgets.interact_manual(tree_number=widgets.IntSlider(min=0, max=rf.get_params(deep=False)["n_estimators"]-1, step=1, value=0), max_depth=widgets.IntSlider(min=0, max=30, step=1, value=2))
def plot_tree(tree_number, max_depth):
    _, ax = plt.subplots()
    tree.plot_tree(rf.estimators_[tree_number],
                   feature_names = feature_list,
                   class_names = labels,
                   filled = True,
                   ax = ax,
                   max_depth=max_depth)
    ax.text(0,1, f"depth = {rf.estimators_[tree_number].get_depth()}")
    plt.show()